# Bayesian Normal Density

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.append('../')

import beer
import numpy as np

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

In [2]:
def plot_normal(fig, mean, cov, n_sigma=2, alpha=1., color='blue'):
    'Plot a Normal density'
    # Eigenvalue decomposition of the covariance matrix.
    evals, evecs = np.linalg.eigh(cov)
    
    sign = 1 if cov[1, 0] == 0 else np.sign(cov[1, 0])
    # Angle of the rotation.
    angle =  - np.arccos(sign * abs(evecs[0, 0]))
   
    fig.ellipse(x=mean[0], y=mean[1], 
                width=n_sigma * 2* np.sqrt(evals[0]), 
                height=n_sigma * 2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)
    
    fig.ellipse(x=mean[0], y=mean[1], 
                width=2 * np.sqrt(evals[0]), 
                height=2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)

Generate some normally distributed data:

In [76]:
mean = np.array([-1.5, 4]) 
cov = np.array([
    [2, 1],
    [1, .75]
])
data = np.random.multivariate_normal(mean, cov, size=1000)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plot_normal(fig, mean, cov, alpha=0.1)

show(fig)

In [77]:
normal_diag = beer.NormalDiagonalCovariance.create(dim=2, prior_count=1e-6)
normal_diag.fit(data)
normal = beer.NormalFullCovariance.create(dim=2, mean=mean, prior_count=1e-6)
normal.fit(data)

In [78]:
print(normal.posterior.grad_lognorm())
print(normal.cov)
print(normal.mean)

[ -0.72662934   0.94378661   0.94378661  -1.88095217  -9.74007252
  17.88062455 -43.07809842   0.32050184]
[[ 1.97569938  0.99132697]
 [ 0.99132697  0.76323106]]
[-1.51790977  3.9914515 ]


In [79]:
fig = figure(
    title='Initial model',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1], alpha=.1)
plot_normal(fig, mean, cov, alpha=0.1)

plot_normal(fig, normal_diag.mean, normal_diag.cov, alpha=.3, color='red')
plot_normal(fig, normal.mean, normal.cov, alpha=.3, color='green')

show(fig)

In [91]:
evals, evecs = np.linalg.eigh(normal.cov)
basis = np.zeros(2)
basis[-1] = evals[-1]
basis = evals

normal2 = beer.NormalFullCovariance.create(dim=2, mean=mean + evecs.T @  np.sqrt(basis), cov=normal.cov.T, prior_count=1e-6)
normal3 = beer.NormalFullCovariance.create(dim=2, mean=mean - evecs.T @  np.sqrt(basis), cov=normal.cov.T, prior_count=1e-6)

In [92]:
fig = figure(
    title='Initial model',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1], alpha=.1)
plot_normal(fig, mean, cov, alpha=0.1)

plot_normal(fig, normal.mean, normal.cov, alpha=.3, color='red')
plot_normal(fig, normal2.mean, normal2.cov, alpha=.3, color='green')
plot_normal(fig, normal3.mean, normal3.cov, alpha=.3, color='purple')


show(fig)

In [90]:
evals, evecs = np.linalg.eigh(normal_diag.cov)
#basis = np.zeros(2)
#basis[-1] = evals[-1]
basis = evals
print(normal.prior.natural_params)


normal2 = beer.NormalDiagonalCovariance.create(dim=2, mean=mean + evecs.T @  np.sqrt(basis), cov=normal_diag.cov, prior_count=1)
normal3 = beer.NormalDiagonalCovariance.create(dim=2, mean=mean - evecs.T @  np.sqrt(basis), cov=normal_diag.cov, prior_count=1)

[  1.00000225e+00  -6.00000000e-06  -6.00000000e-06   1.00001600e+00
  -1.50000000e-06   4.00000000e-06   1.00000000e-06  -9.99999000e-01]


In [86]:
print(normal_diag.cov)
print(normal2.cov)
print(normal3.cov)


[[ 1.97667508  0.        ]
 [ 0.          0.76299431]]
[[ 1.97667508  0.        ]
 [ 0.          0.76299431]]
[[ 1.97667508  0.        ]
 [ 0.          0.76299431]]


In [87]:
fig = figure(
    title='Initial model',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1], alpha=.1)

plot_normal(fig, normal_diag.mean, normal_diag.cov, alpha=.3, color='red')
plot_normal(fig, normal2.mean, normal2.cov, alpha=.3, color='green')
plot_normal(fig, normal3.mean, normal3.cov, alpha=.3, color='purple')


show(fig)

# Model comparison

We generate data for various corrleation parameters:

$$
X_{\lambda} \sim \mathcal{N}(
    \begin{pmatrix} 
    0 \\
    0
    \end{pmatrix}, 
    \begin{pmatrix} 
    1 & \lambda \\
    \lambda & 1
    \end{pmatrix})
$$

and we compare the model evidence for both the Normal distribution with diagonal covariance matrix and with full covariance matrix.

$$
\ln B_{\lambda} = \ln \frac{p(X_\lambda | \mathcal{M}_{\text{full}})}{p(X_\lambda | \mathcal{M}_{\text{diag}})} =
     \ln \frac{\int_{\theta} p(X_\lambda | \theta, \mathcal{M}_{\text{full}}) p(\theta) d\theta}{\int_{\theta}p(X_\lambda | \theta, \mathcal{M}_{\text{diag}})p(\theta) d\theta} = \frac{A_{\text{full}}(\xi + \sum_{n=1}^N T(x_n)) - A_{\text{full}}(\xi)}{A_{\text{diag}}(\xi + \sum_{n=1}^N T(x_n))
     - A_{\text{diag}}(\xi)}
$$

In [8]:
from scipy.special import logsumexp
import copy

lambdas = np.linspace(-.99, .99, 100)
lBs = []

# For each value of lambda.
for l in lambdas:
    # Generate the data.
    cov = np.array([
        [1, l],
        [l, 1]
    ])
    X = np.random.multivariate_normal(np.zeros(2), cov, size=1000)

    # Fit both models
    normal_diag = beer.NormalDiagonalCovariance.create(dim=2, prior_count=1e-6)
    normal_diag.fit(X)
    normal = beer.NormalFullCovariance.create(dim=2, mean=mean, prior_count=1e-6)
    normal.fit(X)
    
    # Compute the log Bayes factor.
    llh_M1 = normal.posterior.lognorm() - normal.prior.lognorm()
    llh_M2 = normal_diag.posterior.lognorm() - normal_diag.prior.lognorm()
    lBs.append(llh_M1 - llh_M2)

fig1 = figure(
    title='Model Comparison',
    x_axis_label='λ',
    y_axis_label='log Bayes factor',
    width=400,
    height=400
)
fig1.line(lambdas, lBs)

fig2 = figure(
    title='Models\' posterior distribution.',
    x_axis_label='λ',
    y_axis_label='p(M|X)',
    width=400,
    height=400
)
fig2.line(lambdas, model_posts[:, 0])
fig2.line(lambdas, model_posts[:, 1], color='green')

#grid = gridplot([[fig1, fig2]])
#show(grid)

show(fig1)